In [ ]:
pip install neo4j

In [ ]:
pip install pandas

In [ ]:
import pandas as pd
import os

In [ ]:
from neo4j import GraphDatabase

# Replace these with your actual AuraDB connection details
uri = "neo4j+s://b480f347.databases.neo4j.io"
username = "neo4j"
password = "8erltLOKbpg68_LbDjbZtsHQC-nAKV-advvPvHLdpII"

driver = GraphDatabase.driver(uri, auth=(username,password))


In [ ]:
from google.colab import drive  # Mounting drive to Google Colab. Ignore if not using Colab
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def create_nodes(tx, data, label, identifier_key):
    for index, row in data.iterrows():
        node_data = dict(row)
        tx.run(
            f"CREATE (node:{label} {{{identifier_key}: $value}})",
            value=node_data[identifier_key]
        )

def create_nodes_spdx(tx, data, label, identifier_key):
    for index, row in data.iterrows():
        node_data = dict(row)
        # Create the node with properties from the CSV columns
        query = (
            f"MERGE (node:{label} {{{identifier_key}: $value}}) "
            f"ON CREATE SET "
            f"node.SPDXID = $SPDXID, "
            f"node.FileName = $FileName, "
            f"node.FileChecksum = $FileChecksum, "
            f"node.LicenseConcluded = $LicenseConcluded, "
            f"node.LicenseInfoInFile = $LicenseInfoInFile, "
            f"node.FileCopyrightText = $FileCopyrightText, "
            f"node.folder_connected = false "
            # Add more ON CREATE property assignments as needed
            f"ON MATCH SET "
            f"node.folder_connected = false"
        )
        tx.run(query, {
            'value': node_data[identifier_key],
            'SPDXID': node_data['SPDXID'],
            'FileName': node_data['FileName'],
            'FileChecksum': node_data['FileChecksum'],
            'LicenseConcluded': node_data['LicenseConcluded'],
            'LicenseInfoInFile': node_data['LicenseInfoInFile'],
            'FileCopyrightText': node_data['FileCopyrightText']
            # Add more property values as needed
        })

def create_nodes_packages(tx, data, label, identifier_key):
    for index, row in data.iterrows():
        node_data = dict(row)
        # Create the node with properties from the CSV columns
        query = (
            f"MERGE (node:{label} {{{identifier_key}: $value}}) "
            f"ON CREATE SET "
            f"node.SPDXID = $SPDXID, "
            f"node.PackageName = $PackageName, "
            f"node.PackageDownloadLocation = $PackageDownloadLocation, "
            f"node.FilesAnalyzed = $FilesAnalyzed, "
            f"node.PackageLicenseConcluded = $PackageLicenseConcluded, "
            f"node.PackageLicenseDeclared = $PackageLicenseDeclared, "
            f"node.PackageCopyrightText = $PackageCopyrightText, "
            f"node.PackageChecksum = $PackageChecksum, "
            f"node.folder_connected = false "
            # Add more ON CREATE property assignments as needed
            f"ON MATCH SET "
            f"node.folder_connected = false"
        )
        tx.run(query, {
            'value': node_data[identifier_key],
            'SPDXID': node_data['SPDXID'],
            'PackageName': node_data['PackageName'],
            'PackageDownloadLocation': node_data['PackageDownloadLocation'],
            'FilesAnalyzed': node_data['FilesAnalyzed'],
            'PackageLicenseConcluded': node_data['PackageLicenseConcluded'],
            'PackageLicenseDeclared': node_data['PackageLicenseDeclared'],
            'PackageCopyrightText': node_data['PackageCopyrightText'],
            'PackageChecksum': node_data['PackageChecksum']
        })


In [ ]:
# Specify the root directory containing subfolders
root_directory = '/content/drive/MyDrive/Cloud Computing EC528/test'

# Iterate through subfolders
for folder_name in os.listdir(root_directory):
    folder_path = os.path.join(root_directory, folder_name)

    if os.path.isdir(folder_path):
        print(f"Processing folder: {folder_name}###############################################################")

        driver = GraphDatabase.driver(uri, auth=(username,password))

        # Load the CSV files for this folder
        packages_data = pd.read_csv(os.path.join(folder_path, f'{folder_name}_packages_output.csv'))
        spdx_data = pd.read_csv(os.path.join(folder_path, f'{folder_name}_spdx_output.csv'))
        relationships_data = pd.read_csv(os.path.join(folder_path, f'{folder_name}_relationships_output.csv'))


        with driver.session() as session:
            # Create nodes and relationships for this subfolder
            print(f"Creating nodes and relationships for: {folder_name}")
            session.write_transaction(create_nodes_spdx, spdx_data, 'SPDXNode', 'SPDXID')
            session.write_transaction(create_nodes_packages, packages_data, 'PackageNode', 'SPDXID')

            for index, row in relationships_data.iterrows():
                relationship_data = dict(row)
                session.run(
                    "MATCH (source {SPDXID: $source}), (target {SPDXID: $target}) "
                    "CREATE (source)-[:RELATIONSHIP_TYPE {type: $types}]->(target)",
                    source=relationship_data['RelationshipType'],
                    target=relationship_data['Target'],
                    types=relationship_data['Source']
                )

 #       # Create a node for the folder and connect all nodes to it
        with driver.session() as session:
            print(f"Creating folder node for: {folder_name}")
            session.run("CREATE (folder:Folder {name: $folder_name})", folder_name=folder_name)

        with driver.session() as session:
            # Connect all nodes in this subfolder to the folder
            for label, identifier_key in [('SPDXNode', 'SPDXID'), ('PackageNode', 'SPDXID')]:
                print(f"Connecting {label} nodes to folder node for: {folder_name}")
                session.run(
                    f"MATCH (node:{label}) WHERE NOT node:Folder AND node.folder_connected = false "
                    "WITH node "
                    "MATCH (folder:Folder {name: $folder_name}) "
                    "CREATE (node)-[:BELONGS_TO]->(folder)"
                    "SET node.folder_connected = true",  # Use a flag to mark nodes as connected
                    label=label,
                    folder_name=folder_name
                )




Processing folder: v1.19.14###############################################################
Creating nodes and relationships for: v1.19.14


<ipython-input-15-0ec027aa3473>:22: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_nodes_spdx, spdx_data, 'SPDXNode', 'SPDXID')
<ipython-input-15-0ec027aa3473>:23: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_nodes_packages, packages_data, 'PackageNode', 'SPDXID')


Creating folder node for: v1.19.14
Connecting SPDXNode nodes to folder node for: v1.19.14
Connecting PackageNode nodes to folder node for: v1.19.14
Processing folder: v1.19.12###############################################################
Creating nodes and relationships for: v1.19.12


<ipython-input-15-0ec027aa3473>:22: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_nodes_spdx, spdx_data, 'SPDXNode', 'SPDXID')
<ipython-input-15-0ec027aa3473>:23: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_nodes_packages, packages_data, 'PackageNode', 'SPDXID')


Creating folder node for: v1.19.12
Connecting SPDXNode nodes to folder node for: v1.19.12
Connecting PackageNode nodes to folder node for: v1.19.12
Processing folder: v1.20.12###############################################################
Creating nodes and relationships for: v1.20.12


...

Creating folder node for: v1.20.10
Connecting SPDXNode nodes to folder node for: v1.20.10
Connecting PackageNode nodes to folder node for: v1.20.10
Processing folder: v1.20.8###############################################################
Creating nodes and relationships for: v1.20.8


<ipython-input-15-0ec027aa3473>:22: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_nodes_spdx, spdx_data, 'SPDXNode', 'SPDXID')
<ipython-input-15-0ec027aa3473>:23: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_nodes_packages, packages_data, 'PackageNode', 'SPDXID')


Creating folder node for: v1.20.8
Connecting SPDXNode nodes to folder node for: v1.20.8
Connecting PackageNode nodes to folder node for: v1.20.8
Processing folder: v1.20.9###############################################################
Creating nodes and relationships for: v1.20.9


<ipython-input-15-0ec027aa3473>:22: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_nodes_spdx, spdx_data, 'SPDXNode', 'SPDXID')
<ipython-input-15-0ec027aa3473>:23: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_nodes_packages, packages_data, 'PackageNode', 'SPDXID')


Creating folder node for: v1.20.9
Connecting SPDXNode nodes to folder node for: v1.20.9
Connecting PackageNode nodes to folder node for: v1.20.9
